In [7]:
###########################################################################
#% FUNCTION:
#% WRITER: SOUMITRA SAMANTA            DATE: 21022019
#% For bug and others mail me at soumitramath39@gmail.com
#%--------------------------------------------------------------------------
#% INPUT:
#% OUTPUT:
#%---------------------------------------------------------------------------
#% EXAMPLE:
#%
##########################################################################
# Some header and necessary function import
#%reset -f
import numpy as np
import time
import matplotlib as plt
from IPython.display import clear_output
from sklearn.preprocessing import normalize
import scipy.io as sio
##############################################################################################

import os
import sys

ss_lib_path = 'SS_PYLIBS/'
sys.path.insert(0, ss_lib_path)
#---------------------------------------------------------------------------------------------

from ss_computation import *
from ss_image_video import *
from ss_drawing import *
from ss_input_output import *
from ss_object_tracking import *
from ss_human_action_data_gen_helper import *
from ss_action_heuristics import *
##############################################################################################

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
#%load_ext autoreload
#%autoreload 2
##############################################################################################



In [46]:
input_data_path = '/Users/soumitra/Documents/SS_DATASET/SS_VIDEO/SS_LUCID/Franklin/animations/revised/tracked_info/'

input_pos_file_extention = 'mat'
input_video_file_extention = 'mp4'
ougput_video_file_extention = 'mp4'

input_pos_file_info = glob.glob('' .join([input_data_path, '*.', input_pos_file_extention]))
input_pos_file_info = sorted(input_pos_file_info)
num_pos_files = len(input_pos_file_info)

input_video_file_info = glob.glob('' .join([input_data_path, 'videofile/*.', input_video_file_extention]))
input_video_file_info = sorted(input_video_file_info)
num_video_files = len(input_video_file_info)

if num_pos_files==num_video_files:
    for snpf in range(num_pos_files):
        print('processing {} ({}/{})' .format(input_pos_file_info[snpf], snpf, num_pos_files))
        mat_contents = sio.loadmat(input_pos_file_info[snpf])
        pos = mat_contents['pos'].astype(float)

        images = ss_video2images(input_video_file_info[snpf])

        num_objects, num_frames= pos.shape[1], pos.shape[2]

        # action heuristics
        object_radius = 10
        sf = 100

        delay_info, interval_delay_info =  ss_Heuristic_DelayCountInterval(pos)
        relative_distance = ss_Heuristic_RelativeDistance(pos)
        relative_angle_of_motion = np.exp(-3.0*(ss_Heuristic_RelativeAngleOfMotion(pos)/180.))
        acceleration_magnitude_interval = ss_Heuristic_AvgAccelerationMagnitudeInterval(pos, 1)
        aveVelMag = fc_heuristic_averageVelocityMagnitude(pos, end_frame=20)# 60

        Temnp_acceleration_magnitude_interval = np.zeros((acceleration_magnitude_interval.shape[0], delay_info.shape[1]))
        Temnp_acceleration_magnitude_interval[:,Temnp_acceleration_magnitude_interval.shape[1]-acceleration_magnitude_interval.shape[1]:] = acceleration_magnitude_interval
        Temnp_acceleration_magnitude_interval[Temnp_acceleration_magnitude_interval<=0.0] = np.finfo(float).eps
        Temnp_acceleration_magnitude_interval = np.exp(-(1./Temnp_acceleration_magnitude_interval))

        Temnp_aveVelMag = aveVelMag.copy()
        Temnp_aveVelMag[Temnp_aveVelMag<=0.0] = np.finfo(float).eps
        Temnp_aveVelMag = np.exp(-(1./Temnp_aveVelMag))

        Temp_delay_info = np.exp(-(delay_info.astype(float)))
        Temp_interval_delay_info = np.exp(-(interval_delay_info+0.9))#/interval_thgreh))

        Temp_relative_distance = relative_distance.copy()
        Temp_relative_distance[xrange(num_points),xrange(num_points),:] = relative_distance.max()
#         Temp_relative_distance = Temp_relative_distance-(2.*object_radius+2)
#         Temp_relative_distance = Temp_relative_distance*(Temp_relative_distance>0.0)
        Temp_relative_distance = Temp_relative_distance.astype(float)/(sf+1.)
        Temp_relative_distance = np.exp(-(Temp_relative_distance))
        
        # combined all heuristics
        est_action_score = np.zeros(relative_distance.shape)
        for i in range(num_objects):
            for j in range(num_objects):
                est_action_score[i,j, :] = Temp_relative_distance[i,j, :]*relative_angle_of_motion[i,j,:]*Temnp_aveVelMag[i,:]*Temnp_acceleration_magnitude_interval[j,:]#231118
            
    
        # for visualization    
        #for object label
        text_id = ss_arrayelements2string(np.array(range(num_objects)))
        num_frames = len(images)
        # draw object label in each video frames
        for snt in xrange(num_frames):
            images[snt] = ss_draw_textOnImage(images[snt], text_id, np.reshape(pos[[1, 0],:,snt],(pos.shape[0], pos.shape[1])))

        # add the cost matrix
        cost_image_size = [20*num_objects, 20*num_objects]
        num_frames = est_action_score.shape[2]

        title_image = ss_create_imageWithText(['new algo score'], text_pos=np.array([[10],[50]]), text_font_size=0.6, text_width=1, text_color=np.array([[255],[0],[0]]))

        for i in range(num_frames):
            cost_img_RD = ss_image_create_cost_matrix(Temp_relative_distance[:,:,i], cost_image_size=cost_image_size, display_flag='area', title='RD')
            cost_img_RAOM = ss_image_create_cost_matrix(relative_angle_of_motion[:,:,i], cost_image_size=cost_image_size, display_flag='area', title='RAM')
            
            cost_img_AVM = ss_image_create_cost_matrix(np.reshape(Temnp_aveVelMag[:,i], (Temnp_aveVelMag.shape[0], 1)), cost_image_size=[cost_image_size[0], 2*(cost_image_size[1]/num_objects)], display_flag='area', title='AvgVelMag')
            cost_img_AAM = ss_image_create_cost_matrix(np.reshape(Temnp_acceleration_magnitude_interval[:,i], (Temnp_acceleration_magnitude_interval.shape[0], 1)), cost_image_size=[cost_image_size[0], 2*(cost_image_size[1]/num_objects)], display_flag='area', title='AvgAccMag')
            
            
            cost_img_PS = ss_image_create_cost_matrix(est_action_score[:,:,i], cost_image_size=cost_image_size, display_flag='area', title='comb. score')

            frame_no_image = ss_create_imageWithText(['f#:{} ({})' .format(i, relative_distance[0, 1, i])], row=[], column=200, text_pos=np.array([[10],[40]]), text_font_size=0.6, text_width=1, text_color=np.array([[255],[0],[0]]))


            cost_img = ss_images_concatenate((ss_images_concatenate((cost_img_RD, cost_img_RAOM, cost_img_PS), axis=0), ss_images_concatenate((cost_img_AVM, cost_img_AAM, frame_no_image), axis=0), cost_img_DI), axis=1)

            cost_img = ss_images_concatenate((title_image, cost_img), axis=0)
            Temp_border_image = 255*np.ones((cost_img.shape[0], 20, cost_img.shape[2]), np.uint8)
            images[i] = ss_images_concatenate((images[i], Temp_border_image, cost_img, Temp_border_image), axis=1)

        # save the video with the cost matrix
        ss_images2video(0, images, save_folder='video_301018/', output_file=input_video_file_info[snpf].split('/')[-1].split('.')[0], output_file_format=ougput_video_file_extention)# for video without label info

else:
    print('# {} mat files is not same as # {} video files'  .format(num_pos_files, num_video_files))
                        



processing /Users/soumitra/Documents/SS_DATASET/SS_VIDEO/SS_LUCID/Franklin/animations/revised/tracked_info/tracked_pos_chase.mat (0/7)
processing /Users/soumitra/Documents/SS_DATASET/SS_VIDEO/SS_LUCID/Franklin/animations/revised/tracked_info/tracked_pos_coaxing.mat (1/7)
processing /Users/soumitra/Documents/SS_DATASET/SS_VIDEO/SS_LUCID/Franklin/animations/revised/tracked_info/tracked_pos_fighting.mat (2/7)
processing /Users/soumitra/Documents/SS_DATASET/SS_VIDEO/SS_LUCID/Franklin/animations/revised/tracked_info/tracked_pos_leading.mat (3/7)
processing /Users/soumitra/Documents/SS_DATASET/SS_VIDEO/SS_LUCID/Franklin/animations/revised/tracked_info/tracked_pos_mocking.mat (4/7)
processing /Users/soumitra/Documents/SS_DATASET/SS_VIDEO/SS_LUCID/Franklin/animations/revised/tracked_info/tracked_pos_seducing.mat (5/7)
processing /Users/soumitra/Documents/SS_DATASET/SS_VIDEO/SS_LUCID/Franklin/animations/revised/tracked_info/tracked_pos_suprising.mat (6/7)
